# Sudoku Search for Training Data

In [3]:
from platform import python_version

print(python_version())

3.10.0


In [4]:
import os
import uuid
import hashlib
import torch
import ray
from fastbook import *

In [5]:
imageDir = "./train/images"

In [8]:
hashCache = []

if ray.is_initialized() == False:
    ray.init(include_dashboard=False)

2023-12-18 23:58:39,837	INFO worker.py:1621 -- Started a local Ray instance.
(raylet) [2023-12-18 23:58:55,987 E 283 319] (raylet) agent_manager.cc:135: The raylet exited immediately because the Ray agent failed. The raylet fate shares with the agent. This can happen because the Ray agent was unexpectedly killed or failed. Agent can fail when
(raylet) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/dashboard_agent.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure.
(raylet) - The agent is killed by the OS (e.g., out of memory).


In [9]:
urls = search_images_ddg('sudoku', max_images=300)
len(urls),urls[0]

(300,
 'https://sudokuprintables.com/wp-content/uploads/2020/04/sudoku-puzzles.jpg')

In [10]:
def calculateMd5(filename):
    runningMd5 = hashlib.md5()
    with open(filename, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            runningMd5.update(chunk)
    return runningMd5.hexdigest()

In [11]:
def doesContentAlreadyExist(filename, dir):
    newFileHash = calculateMd5(filename)

    if (len(hashCache) == 0):
        for f in os.listdir(dir):
            absFile = os.path.join(dir, f)
            if os.path.isfile(absFile):
                hash = calculateMd5(absFile)
                hashCache.append(hash)

    for hash in hashCache:
        if (hash == newFileHash):
            return True
            
    return False

In [12]:
for url in urls:
    url = url.split('?')[0]
    print ("Processing URL: " + url)
    e = re.findall('\.[a-z]+$', url)
    if (len(e)) == 1:
        fileExtension = e[0]
        randomName = str(uuid.uuid4().hex)
        filename = randomName + fileExtension
        tempFilename = "/tmp/" + filename
        
        download_url(url, tempFilename)

        if doesContentAlreadyExist(tempFilename, imageDir) == True:
            print ("Found: " + tempFilename)
            os.remove(tempFilename)
        else:
            print ("NEW: " + tempFilename)
            os.replace(tempFilename, imageDir + "/" + filename)

    else:
        print ("Unable to process image due to unexpected extension: " + url)

Processing URL: https://sudokuprintables.com/wp-content/uploads/2020/04/sudoku-puzzles.jpg


NEW: /tmp/dfa8287f8c11481ea504cf1ec70b175f.jpg


OSError: [Errno 18] Invalid cross-device link: '/tmp/dfa8287f8c11481ea504cf1ec70b175f.jpg' -> './train/images/dfa8287f8c11481ea504cf1ec70b175f.jpg'

In [13]:
ds = ray.data.read_images(imageDir)
ds.size_bytes()


LocalRayletDiedError: The task's local raylet died. Check raylet.out for more information.